In [3]:
# raw data on April 1, 2022 from 16:10 to 16:40 UTC. Launch from SLC40 at 16:24 UTC, watched from Titusville
import os, sys, glob, obspy
import numpy as np
import pandas as pd
cwd = os.getcwd()
sys.path.append(os.path.join(cwd, 'campbell'))
import read_cs_files as campbell

HOME = os.getenv('HOME')
SDS_TOP = os.path.join(HOME, 'Desktop/Desktop - Glenn’s MacBook Air/KSC_new')
DROPBOX_TOP = os.path.join(HOME, 'Dropbox')
DROPBOX_PROJECT_DIR = os.path.join(DROPBOX_TOP, 'PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion')
EVENT_MSEED_DIR = os.path.join(DROPBOX_PROJECT_DIR, 'KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events')
HTML_DIR = './html'
PLOTS_DIR = HTML_DIR

WELLDATA_TOP = os.path.join(DROPBOX_PROJECT_DIR, 'KSC_Well_Seismoacoustic_Data/WellData')
TOB3_DIR = os.path.join(WELLDATA_TOP, 'Uploads')
WELLMSEED_DIR = os.path.join(WELLDATA_TOP, 'miniseed')


def localtime2utc(this_dt):
    localTimeCorrection = 3600 * 4
    return this_dt + localTimeCorrection

# Read Campbell Scientific TOB3 file
allcolumns = []
tob3files = glob.glob(os.path.join(TOB3_DIR, '2022????/*.dat'))
lod = []
for tob3file in tob3files:
    data, meta = campbell.read_cs_files(tob3file, forcedatetime=False,
                      bycol=True, quiet=True, metaonly=False)

    df = pd.DataFrame(columns=meta[2])
    for c in range(len(meta[2])):
        df[meta[2][c]] = data[c]
        if not meta[2][c] in allcolumns:
            allcolumns.append(meta[2][c])
          
    # convert to Miniseed
    local_startt = obspy.UTCDateTime(df.iloc[0]['TIMESTAMP'])
    nextt = obspy.UTCDateTime(df.iloc[1]['TIMESTAMP'])
    dt = nextt-local_startt
    utc_startt = localtime2utc(local_startt)
    mseedfile = os.path.join(WELLMSEED_DIR, '%s.%s.mseed' % (utc_startt.strftime('%Y%m%d%H%M%S'), tr.id  ) )
    if os.path.exists(mseedfile): # already exists. skipping.
        print('%s already exists. skipping.' % mseedfile)
        continue

    tob3base = os.path.basename(tob3file)
    if tob3base[:3]=='HOF':
        sta = tob3base[9:11]
    elif tob3base[:8]=='Absolute':
        sta = 'AB'
    
    for this_col in df.columns[2:]:
        tr = obspy.Trace()
        tr.id = 'FL.S39%s.00.' % sta
        if this_col[1]=='_' and this_col[-1]=='A':
            tr.stats.station = 'AB6%s' % this_col[0] 
        if 'TempC' in this_col:
            tr.stats.channel = 'HKD'
        elif this_col=='Baro':
            tr.stats.channel = 'HDH'
        elif 'psi' in this_col:
            tr.stats.channel = 'HDD'
            if this_col[1]=='_' and this_col[-1]=='A':
                tr.stats.location = this_col[2:4] 
            else:
                tr.stats.location = this_col[:2]
        tr.stats.starttime = utc_startt
        tr.stats.delta = dt
        tr.data = np.array(df[this_col])
        print('Writing %s' % mseedfile)
        tr.write(mseedfile)
        thisrow = {'TOB3file':tob3base, 'miniseedFile':os.path.basename(mseedfile)}
        lod.append(thisrow)
df = pd.DataFrame(lod)
df.to_csv(os.path.join(WELLDATA_TOP,'lookuptable_tob3file_miniseedfile.csv'))

['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA']
../WellMiniseed/20220421160000.FL.AB6I.30.HDD.mseed
../WellMiniseed/20220421160000.FL.AB6S.30.HDD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA']
../WellMiniseed/20220531120000.FL.AB6I.30.HDD.mseed
../WellMiniseed/20220531120000.FL.AB6S.30.HDD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA']
../WellMiniseed/20220602200000.FL.AB6I.30.HDD.mseed
../WellMiniseed/20220602200000.FL.AB6S.30.HDD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA']
../WellMiniseed/20220504120000.FL.AB6I.30.HDD.mseed
../WellMiniseed/20220504120000.FL.AB6S.30.HDD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220523200000.FL.S396S.00.HDH.mseed
../WellMiniseed/20220523200000.FL.S396S.15.HDD.mseed
../WellMiniseed/20220523200000.FL.S396S.70.HDD.mseed
../WellMiniseed/20220523200000.FL.S396S.30.HDD.mseed
../WellMiniseed/20220523200000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA',

['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220530120000.FL.S396S.00.HDH.mseed
../WellMiniseed/20220530120000.FL.S396S.15.HDD.mseed
../WellMiniseed/20220530120000.FL.S396S.70.HDD.mseed
../WellMiniseed/20220530120000.FL.S396S.30.HDD.mseed
../WellMiniseed/20220530120000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220516080000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220516080000.FL.S396I.15.HDD.mseed
../WellMiniseed/20220516080000.FL.S396I.70.HDD.mseed
../WellMiniseed/20220516080000.FL.S396I.30.HDD.mseed
../WellMiniseed/20220516080000.FL.S396I.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220515080000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220515080000.FL.S396I.15.HDD.mseed
../WellMiniseed/20220515080000.FL.S396I.70.HDD.mseed
../WellMiniseed/20220515080000.FL.S

../WellMiniseed/20220523080000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220430040000.FL.AB6I.30.HDD.mseed
../WellMiniseed/20220430040000.FL.AB6S.30.HDD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220514200000.FL.S396S.00.HDH.mseed
../WellMiniseed/20220514200000.FL.S396S.15.HDD.mseed
../WellMiniseed/20220514200000.FL.S396S.70.HDD.mseed
../WellMiniseed/20220514200000.FL.S396S.30.HDD.mseed
../WellMiniseed/20220514200000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220528160000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220528160000.FL.S396I.15.HDD.mseed
../WellMiniseed/20220528160000.FL.S396I.70.HDD.mseed
../WellMiniseed/20220528160000.FL.S396I.30.HDD.mseed
../WellMiniseed/20220528160000.FL.S396I.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', '

../WellMiniseed/20220530000000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220429160000.FL.S396S.00.HDH.mseed
../WellMiniseed/20220429160000.FL.S396S.15.HDD.mseed
../WellMiniseed/20220429160000.FL.S396S.70.HDD.mseed
../WellMiniseed/20220429160000.FL.S396S.30.HDD.mseed
../WellMiniseed/20220429160000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220522000000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220522000000.FL.S396I.15.HDD.mseed
../WellMiniseed/20220522000000.FL.S396I.70.HDD.mseed
../WellMiniseed/20220522000000.FL.S396I.30.HDD.mseed
../WellMiniseed/20220522000000.FL.S396I.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220523000000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220523000000.FL.S396I.15.HDD.mseed
../WellMiniseed/20220523000000.FL.S

../WellMiniseed/20220427040000.FL.S396I.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220525200000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220525200000.FL.S396I.15.HDD.mseed
../WellMiniseed/20220525200000.FL.S396I.70.HDD.mseed
../WellMiniseed/20220525200000.FL.S396I.30.HDD.mseed
../WellMiniseed/20220525200000.FL.S396I.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220527200000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220527200000.FL.S396I.15.HDD.mseed
../WellMiniseed/20220527200000.FL.S396I.70.HDD.mseed
../WellMiniseed/20220527200000.FL.S396I.30.HDD.mseed
../WellMiniseed/20220527200000.FL.S396I.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220429160000.FL.AB6I.30.HDD.mseed
../WellMiniseed/20220429160000.FL.AB6S.30.HDD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', '

...

../WellMiniseed/20220502200000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220430200000.FL.S396S.00.HDH.mseed
../WellMiniseed/20220430200000.FL.S396S.15.HDD.mseed
../WellMiniseed/20220430200000.FL.S396S.70.HDD.mseed
../WellMiniseed/20220430200000.FL.S396S.30.HDD.mseed
../WellMiniseed/20220430200000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220518160000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220518160000.FL.S396I.15.HDD.mseed
../WellMiniseed/20220518160000.FL.S396I.70.HDD.mseed
../WellMiniseed/20220518160000.FL.S396I.30.HDD.mseed
../WellMiniseed/20220518160000.FL.S396I.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220524200000.FL.S396S.00.HDH.mseed
../WellMiniseed/20220524200000.FL.S396S.15.HDD.mseed
../WellMiniseed/20220524200000.FL.S

../WellMiniseed/20220518000000.FL.S396S.30.HDD.mseed
../WellMiniseed/20220518000000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220503160000.FL.AB6I.30.HDD.mseed
../WellMiniseed/20220503160000.FL.AB6S.30.HDD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220525040000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220525040000.FL.S396I.15.HDD.mseed
../WellMiniseed/20220525040000.FL.S396I.70.HDD.mseed
../WellMiniseed/20220525040000.FL.S396I.30.HDD.mseed
../WellMiniseed/20220525040000.FL.S396I.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220501080000.FL.AB6I.30.HDD.mseed
../WellMiniseed/20220501080000.FL.AB6S.30.HDD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220520080000.FL.S396S.00.HDH.mseed


['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220503120000.FL.AB6I.30.HDD.mseed
../WellMiniseed/20220503120000.FL.AB6S.30.HDD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220518040000.FL.S396S.00.HDH.mseed
../WellMiniseed/20220518040000.FL.S396S.15.HDD.mseed
../WellMiniseed/20220518040000.FL.S396S.70.HDD.mseed
../WellMiniseed/20220518040000.FL.S396S.30.HDD.mseed
../WellMiniseed/20220518040000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220525080000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220525080000.FL.S396I.15.HDD.mseed
../WellMiniseed/20220525080000.FL.S396I.70.HDD.mseed
../WellMiniseed/20220525080000.FL.S396I.30.HDD.mseed
../WellMiniseed/20220525080000.FL.S396I.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC'

../WellMiniseed/20220520120000.FL.S396I.30.HDD.mseed
../WellMiniseed/20220520120000.FL.S396I.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220530160000.FL.S396S.00.HDH.mseed
../WellMiniseed/20220530160000.FL.S396S.15.HDD.mseed
../WellMiniseed/20220530160000.FL.S396S.70.HDD.mseed
../WellMiniseed/20220530160000.FL.S396S.30.HDD.mseed
../WellMiniseed/20220530160000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20491202203542.FL.S396S.00.HDH.mseed
../WellMiniseed/20491202203542.FL.S396S.15.HDD.mseed
../WellMiniseed/20491202203542.FL.S396S.70.HDD.mseed
../WellMiniseed/20491202203542.FL.S396S.30.HDD.mseed
../WellMiniseed/20491202203542.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220522160000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220522160000.FL.S

../WellMiniseed/20220503080000.FL.S396S.30.HDD.mseed
../WellMiniseed/20220503080000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220501000000.FL.S396S.00.HDH.mseed
../WellMiniseed/20220501000000.FL.S396S.15.HDD.mseed
../WellMiniseed/20220501000000.FL.S396S.70.HDD.mseed
../WellMiniseed/20220501000000.FL.S396S.30.HDD.mseed
../WellMiniseed/20220501000000.FL.S396S.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220515200000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220515200000.FL.S396I.15.HDD.mseed
../WellMiniseed/20220515200000.FL.S396I.70.HDD.mseed
../WellMiniseed/20220515200000.FL.S396I.30.HDD.mseed
../WellMiniseed/20220515200000.FL.S396I.00.HKD.mseed
['TIMESTAMP', 'RECORD', 'I_30psiA', 'S_30psiA', 'Baro', '15psi', '70psi', '30psi', 'TempC']
../WellMiniseed/20220517200000.FL.S396I.00.HDH.mseed
../WellMiniseed/20220517200000.FL.S

In [ ]:
s = 30000
a = 9.8
t = 300
s = u * t + 0.5 * a * t^2
ax2 + bx + c = 0
a = -4.9
b = u
c = -s
x = -b +/- sqrt(b^2 - 4 * a * c)/(2 * a)


u = (30000 + 4.9 * 300^2)/300 = (100 + 4.9 * 300) = 1600

1600

In [ ]:
s = 28000
a = 9.8
t = 480
s = u * t + 0.5 * a * t^2
ax2 + bx + c = 0
a = -4.9
b = u
c = -s
x = -b +/- sqrt(b^2 - 4 * a * c)/(2 * a)


u = (28000 + 4.9 * 480^2)/480 = (58 + 2352) = 1600

1600

In [4]:
28000/480

58.333333333333336

In [5]:
4.9 * 480

2352.0